# Cross validation and bootstrap lab (R)

From An Introduction to Statistical Learning, chapter 5 lab.

In [51]:
library(ISLR)
library(boot)

set.seed(1)
train = sample(392, 196)

#attach(Auto)

## 1 Validation set

### 1.1 Linear fit

In [43]:
lm.fit = lm(mpg~horsepower, data=Auto, subset=train)
mean((mpg - predict(lm.fit, Auto))[-train]^2)

[1] 23.26601

### 1.2 Quadratic fit

In [44]:
lm.fit2 = lm(mpg~poly(horsepower, 2), data=Auto, subset=train)
mean((mpg - predict(lm.fit2, Auto))[-train]^2)

[1] 18.71646

### 1.3 Cubic fit

In [45]:
lm.fit3 = lm(mpg~poly(horsepower, 3), data=Auto, subset=train)
mean((mpg - predict(lm.fit3, Auto))[-train]^2)

[1] 18.79401

## 2 Leave-one-out cross validation (LOOCV)

### 2.1 Linear fit

In [49]:
glm.fit = glm(mpg~horsepower, data=Auto)
coef(glm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [56]:
cv.err = cv.glm(Auto, glm.fit)
cv.err$delta

[1] 19.03321 19.03242

### 2.2 Degree 1-5 fits

In [54]:
cv.error = rep(0, 5)
for (i in 1:5) {
    glm.fit = glm(mpg~poly(horsepower, i), data=Auto)
    cv.error[i] = cv.glm(Auto, glm.fit)$delta[1]
}
cv.error

[1] 24.23151 19.24821 19.33498 19.42443 19.03321

## 3 k-fold cross validation

In [58]:
set.seed(17)
cv.error.10 = rep(0,10)
for (i in  1:10) {
    glm.fit = glm(mpg~poly(horsepower, i), data=Auto)
    cv.error.10[i] = cv.glm(Auto, glm.fit, K=10)$delta[1]
}
cv.error.10

[1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520

## 4 The bootstrap

### 4.1 Estimating the accuracy of a statistic (alpha)

In [60]:
alpha.fn = function(data, index) {
    X = data$X[index]
    Y = data$Y[index]
    return (var(Y) - cov(X, Y)) / (var(X) + var(Y) - 2 * cov(X, Y))
}

In [61]:
alpha.fn(Portfolio, 1:100)

[1] 0.6818792

In [63]:
set.seed(1)
alpha.fn(Portfolio, sample(100, 100, replace=T))

[1] 0.7748313

In [64]:
boot(Portfolio, alpha.fn, R=1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original      bias    std. error
t1* 0.6818792 -0.01141145   0.1235559

### 4.2 Estimating the accuracy of a linear regression model

In [66]:
boot.fn = function(data, index) {
    return (coef(lm(mpg~horsepower, data=data, subset=index)))
}
boot.fn(Auto, 1:392)

(Intercept)  horsepower 
 39.9358610  -0.1578447

In [67]:
set.seed(1)
boot.fn(Auto, sample(392, 392, replace=T))

(Intercept)  horsepower 
 40.3404517  -0.1634868

In [68]:
boot.fn(Auto, sample(392, 392, replace=T))

(Intercept)  horsepower 
 40.1186906  -0.1577063

In [69]:
boot(Auto, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0544513229 0.841289790
t2* -0.1578447 -0.0006170901 0.007343073

In [70]:
summary(lm(mpg~horsepower, data=Auto))$coef

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81


In [72]:
boot.fn = function(data, index) {
    coefficients(lm(mpg~horsepower+I(horsepower^2), data=data, subset=index))
}

set.seed(1)
boot(Auto, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  3.511640e-02 2.0300222526
t2* -0.466189630 -7.080834e-04 0.0324241984
t3*  0.001230536  2.840324e-06 0.0001172164